In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('summary.csv')

In [ ]:
data.head()

,Unnamed: 0,Headlines,Summary,Companies,Sectors,Intreset
0,0,Atos reaches deal with creditors on debt restr...,French technology company Atos reached an agr...,Atos,Information Technology,0
1,1,Saudi Aramco awards $25 billion in contracts f...,Saudi Aramco has awarded $25 billion in contra...,Saudi Aramco,Energy,0
2,2,Bank of Baroda gets Rs 1067.82 crore income ta...,Bank of Baroda received a tax demand of ₹1067....,Bank of Baroda,Banking,0
3,3,Air India Express union alleges unfair labour ...,"Air India Express’ cabin crew union, AIXEU, ha...",Air India Express,Airlines,0
4,4,"Punjab & Sind Bank plans to raise Rs 2,000 cro...","Punjab & Sind Bank plans to raise ₹2,000 crore...",Punjab & Sind Bank,Banking,1


I am thinking of just converting summary to word indices, since headlines is just a small part of the summary anyways. The companies and sector will be useful for multi-class classification, I dont need them now.

So my X = data['summary'] and y = data['Intreset'] and I guess I will be working on that for now. Will have to ask Alok Mama, if I need anything else for binary classification.

Also I will have to find some site or article which I can web-scrape
( I am probably limited by my own skill level lol)

In [ ]:
data['Intreset'].value_counts(normalize=True)

Intreset
1    0.627907
0    0.372093
Name: proportion, dtype: float64

The data is a bit unbalanced, but I think that wont be a problem, considering how small my dataset is.

This feels more like a trail than the final product.

In [ ]:
X, y = data['Summary'], data['Intreset'].values # I feel it would be better to have an array for y instead of a series

In [ ]:
# This is gonna break my sentences to word
tokenizer = Tokenizer()

# This is gonna create my vocab or the dictionary which maps words to numbers
tokenizer.fit_on_texts(X)

#This creates my word index. An array with numbers, basically a sentence in numbers(if that makes sense?)
word_indices = tokenizer.texts_to_sequences(X)

In [ ]:
print(word_indices[0])

[635, 636, 22, 336, 337, 20, 119, 13, 3, 34, 6, 637, 4, 638, 7, 639, 9, 12, 170, 120, 1, 22, 29, 7, 640, 1, 120, 338, 339, 3, 641, 41, 642, 643, 41, 340, 121, 3, 171, 62, 13, 233, 644, 122, 32, 29, 5, 3, 172, 234, 31, 35, 48, 235, 6, 341, 170, 18, 52, 645, 342, 96, 236, 1, 237, 646, 6, 647, 170, 2, 31, 36, 48, 235, 23, 18, 123, 341, 49, 648, 11, 173, 25, 15, 48, 235, 1, 172, 29, 1, 22, 29, 649, 119, 42, 3, 238, 650, 651, 652, 3, 653, 120, 119, 11, 174, 336, 63, 97, 654, 23, 3, 655, 124, 11, 656, 657, 658, 659, 98, 660, 26, 661, 7, 12, 120]


In [ ]:
vocab = tokenizer.word_index

In [ ]:
max_len = max(max(i) for i in word_indices)
word_indices_padded = pad_sequences(word_indices, maxlen=max_len, padding='post')
word_indices_np_padded = np.array(word_indices_padded)

Ok so, I cant create an numpy array directly as my summaries differ in length (ofc), so chatgpt says I can either using padding ( basically fill the shorter summaries with empty strings to get uniform lengths on every summary). I can do that or create an numpy object array.

I dont know which one is better, will have to ask Alok Mama. <br>
Currenly using object array

In [ ]:
# Creating the vectorize sequence
one_hot_encoded = tokenizer.texts_to_matrix(X, mode='binary')

In [ ]:
one_hot_encoded.shape

(43, 1498)

In [ ]:
y_train = np.asarray(y).astype('float32')

In [ ]:
x_test = word_indices_np_padded[-5:]
y_test = y_train[-5:]

array([1., 0., 1., 1., 0.], dtype=float32)

In [ ]:
word_indices_np_padded = word_indices_np_padded[:-5]
y_train = y_train[:-5]

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(word_indices_np_padded, y_train, epochs=10);

Epoch 1/10
2/2 [==============================] - 1s 15ms/step - loss: 57.2774 - accuracy: 0.4211
Epoch 2/10
2/2 [==============================] - 0s 10ms/step - loss: 17.7037 - accuracy: 0.6316
Epoch 3/10
2/2 [==============================] - 0s 10ms/step - loss: 9.9120 - accuracy: 0.8421
Epoch 4/10
2/2 [==============================] - 0s 14ms/step - loss: 6.5565 - accuracy: 0.8684
Epoch 5/10
2/2 [==============================] - 0s 14ms/step - loss: 4.7042 - accuracy: 0.8947
Epoch 6/10
2/2 [==============================] - 0s 9ms/step - loss: 3.9467 - accuracy: 0.8421
Epoch 7/10
2/2 [==============================] - 0s 10ms/step - loss: 3.9071 - accuracy: 0.8421
Epoch 8/10
2/2 [==============================] - 0s 10ms/step - loss: 3.2846 - accuracy: 0.8684
Epoch 9/10
2/2 [==============================] - 0s 12ms/step - loss: 2.7450 - accuracy: 0.8684
Epoch 10/10
2/2 [==============================] - 0s 11ms/step - loss: 2.2741 - accuracy: 0.9211


In [ ]:
y_pred_prob = model.predict(x_test)

1/1 [==============================] - 0s 93ms/step


In [ ]:
y_pred_prob

array([[0.0000000e+00],
       [3.4031062e-13],
       [6.8153179e-01],
       [4.5460315e-13],
       [8.7038654e-07]], dtype=float32)

In [ ]:
threshold = 0.5

# Convert probabilities to class labels
y_pred_class = (y_pred_prob >= threshold).astype(int)

In [ ]:
accuracy = np.mean(y_pred_class == y_test.reshape(-1, 1))
accuracy

0.6

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred_class)

In [ ]:
cm

array([[2, 0],
       [2, 1]])

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)
roc_auc = roc_auc_score(y_test, y_pred_prob)

In [ ]:
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

Precision: 1.0000
Recall: 0.3333
F1 Score: 0.5000
ROC AUC: 0.5000
